<a href="https://colab.research.google.com/github/leidysguerrero120/Trabajo-Analitica-3---RRHH/blob/main/a_Funciones.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
###Importamos las librerías necesarias
import numpy as np
import pandas as pd
from sklearn.impute import SimpleImputer
from sklearn.feature_selection import SelectFromModel
from sklearn.model_selection import cross_val_score
import joblib
from sklearn.preprocessing import StandardScaler

###Función para ejecutar un archivo SQL que contenga múltiples consultas
def ejecutar_consultas_sql(nombre_archivo, cursor):
    with open(nombre_archivo, 'r') as archivo_sql:
        contenido_sql = archivo_sql.read()
    cursor.executescript(contenido_sql)

###Función para imputar datos numéricos con la mediana y categóricos con la moda
def imputar_datos(df, columnas_categoricas):
    df_cat = df[columnas_categoricas]
    df_num = df.loc[:, ~df.columns.isin(columnas_categoricas)]

    imputador_num = SimpleImputer(strategy='median')
    imputador_cat = SimpleImputer(strategy='most_frequent')

    imputador_num.fit(df_num)
    imputador_cat.fit(df_cat)

    datos_num = imputador_num.transform(df_num)
    datos_cat = imputador_cat.transform(df_cat)

    df_num = pd.DataFrame(datos_num, columns=df_num.columns)
    df_cat = pd.DataFrame(datos_cat, columns=df_cat.columns)

    df = pd.concat([df_num, df_cat], axis=1)
    return df

###Función para seleccionar variables relevantes usando diferentes modelos
def seleccionar_variables(modelos, X, y, umbral):
    nombres_variables = np.array([])
    for modelo in modelos:
        modelo.fit(X, y)
        seleccion = SelectFromModel(modelo, prefit=True, threshold=umbral)
        nombres = modelo.feature_names_in_[seleccion.get_support()]
        nombres_variables = np.append(nombres_variables, nombres)
        nombres_variables = np.unique(nombres_variables)

    return nombres_variables

###Función para evaluar modelos usando validación cruzada
def evaluar_modelos(modelos, metrica, X, y, cv):
    resultados_modelos = pd.DataFrame()
    for modelo in modelos:
        puntuaciones = cross_val_score(modelo, X, y, scoring=metrica, cv=cv)
        resultados = pd.DataFrame(puntuaciones)
        resultados_modelos = pd.concat([resultados_modelos, resultados], axis=1)

    resultados_modelos.columns = ["Regresión_Logística", "Árboles_Decisión", "Bosque_Aleatorio", "Ajuste_Gradiente"]
    return resultados_modelos

###Función para preparar datos: imputación, creación de variables dummy y escalado
def transformar_datos(df):
    columnas_categoricas = joblib.load("columnas_categoricas.pkl")
    columnas_dummies = joblib.load("columnas_dummies.pkl")
    nombres_variables = joblib.load("nombres_variables.pkl")
    escalador = joblib.load("escalador.pkl")

    df = imputar_datos(df, columnas_categoricas)
    df_dummies = pd.get_dummies(df, columns=columnas_dummies)
    df_dummies = df_dummies.loc[:, ~df_dummies.columns.isin(['perf_2023', 'EmpID2'])]
    datos_escalados = escalador.transform(df_dummies)
    X = pd.DataFrame(datos_escalados, columns=df_dummies.columns)
    X = X[nombres_variables]

    return X

###Función para imputar valores nulos con la moda de la variable
def imputar_por_moda(df, variables):
    for variable in variables:
        moda = df[variable].mode()[0]
        df[variable].fillna(moda, inplace=True)
        nulos_restantes = df[variable].isnull().sum()

    return df
